# <center>MobileNet - Pytorch

# Step 1: Prepare data

In [1]:
# MobileNet-Pytorch
import argparse 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score
#from mobilenets import mobilenet

use_cuda = torch.cuda.is_available()
use_cudause_cud  = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

In [2]:
# Train, Validate, Test. Heavily inspired by Kevinzakka https://github.com/kevinzakka/DenseNet/blob/master/data_loader.py

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

valid_size=0.1

# define transforms
valid_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])


# load the dataset
train_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=train_transform)

valid_dataset = datasets.CIFAR10(root="data", train=True, 
            download=True, transform=valid_transform)

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train)) #5w张图片的10%用来当做验证集


np.random.seed(42)
np.random.shuffle(indices) # 随机乱序[0,1,...,49999]

train_idx, valid_idx = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_idx) # 这个很有意思
valid_sampler = SubsetRandomSampler(valid_idx)


# ------------------------- 使用不同的批次大小 ------------------------------------
show_step=10  # 批次大，show_step就小点
max_epoch=60  # 训练最大epoch数目
train_loader = torch.utils.data.DataLoader(train_dataset, 
                batch_size=256, sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                batch_size=256, sampler=valid_sampler)


test_transform = transforms.Compose([
    transforms.ToTensor(), normalize
])

test_dataset = datasets.CIFAR10(root="data", 
                                train=False, 
                                download=True,transform=test_transform)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=64, 
                                          shuffle=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Step 2: Model Config

In [3]:
# 32  缩放5次到 1x1@1024 
# From https://github.com/kuangliu/pytorch-cifar 
import torch
import torch.nn as nn
import torch.nn.functional as F

class Block_Attention(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block_Attention, self).__init__()
        
        # 分组卷积数=输入通道数
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        
        self.bn1 = nn.BatchNorm2d(in_planes)
        
        
        #self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        
        one_conv_kernel_size = 3
        self.conv1D= nn.Conv1d(1, out_planes, one_conv_kernel_size, stride=1,padding=1,groups=1,dilation=1,bias=False) # 在__init__初始化        
        
        self.bn2 = nn.BatchNorm2d(out_planes)
        
        self.Fixed_Random_Vector=torch.randn(256,1,1024).cuda()*0.1 # [bs,1，in_channel] 

    def forward(self, x):
        
        out = F.relu(self.bn1(self.conv1(x)))
        
        # -------------------------- Attention -----------------------
        w = self.Fixed_Random_Vector[0:out.shape[0]] # [bs,1，in_channel] 
        #print(w[0][0])
        #w = F.avg_pool2d(x,x.shape[-1])  #最好在初始化层定义好
        #print(w.shape)
        # [bs,in_Channel,1,1]
        #w = w.view(w.shape[0],1,w.shape[1])
        # [bs,1,in_Channel]
        # one_conv_filter = nn.Conv1d(1, out_channel, one_conv_kernel_size, stride=1,padding=1,groups=1,dilation=1) # 在__init__初始化
        # [bs,out_channel,in_Channel]
        w = self.conv1D(w)
        w = 0.5*F.tanh(w) # [-0.5,+0.5]
        # -------------- softmax ---------------------------
        #print(w.shape)
        w = w.view(w.shape[0],w.shape[1],w.shape[2],1,1)
        #print(w.shape)
        
        # ------------------------- fusion --------------------------
        out=out.view(out.shape[0],1,out.shape[1],out.shape[2],out.shape[3])
        #print("x size:",out.shape)
        
        out=out*w
        #print("after fusion x size:",out.shape)
        out=out.sum(dim=2)
        
        out = F.relu(self.bn2(out))
        
        return out


class Block(nn.Module):
    '''Depthwise conv + Pointwise conv'''
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        
        # 分组卷积数=输入通道数
        self.conv1 = nn.Conv2d(in_planes, in_planes, kernel_size=3, stride=stride, padding=1, groups=in_planes, bias=False)
        
        self.bn1 = nn.BatchNorm2d(in_planes)
        
        self.conv2 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        
        self.bn2 = nn.BatchNorm2d(out_planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        return out


class MobileNet(nn.Module):
    # (128,2) means conv planes=128, conv stride=2, by default conv stride=1
    #cfg = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, 512, (1024,2), 1024]
    cfg = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, 512, (1024,2), [1024,1]]
    
    def __init__(self, num_classes=10):
        super(MobileNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32) # 自动化构建层
        self.linear = nn.Linear(1024, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for x in self.cfg:
            if isinstance(x, int):
                out_planes = x
                stride = 1 
                layers.append(Block(in_planes, out_planes, stride))
            elif isinstance(x, tuple):
                out_planes = x[0]
                stride = x[1]
                layers.append(Block(in_planes, out_planes, stride))
            # AC层通过list存放设置参数
            elif isinstance(x, list):
                out_planes= x[0]
                stride = x[1] if len(x)==2 else 1
                layers.append(Block_Attention(in_planes, out_planes, stride))   
            else:
                pass
            
            in_planes = out_planes
            
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [4]:
# From https://github.com/Z0m6ie/CIFAR-10_PyTorch
#model = mobilenet(num_classes=10, large_img=False)

# From https://github.com/kuangliu/pytorch-cifar 
if torch.cuda.is_available():
    model=MobileNet(10).cuda()
else:
    model=MobileNet(10)

optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss()

In [5]:
# Implement validation
def train(epoch):
    model.train()
    #writer = SummaryWriter()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        correct = 0
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()
        
        loss = criterion(output, target)
        loss.backward()
        accuracy = 100. * (correct.cpu().numpy()/ len(output))
        optimizer.step()
        if batch_idx % show_step == 0:
            if batch_idx % 16*show_step==0:
                print(torch.abs(model.layers[12].conv1D.weight).sum())
                print(model.layers[12].conv1D.weight.shape)
                print(model.layers[12].conv1D.weight)
                
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), accuracy))
            #writer.add_scalar('Loss/Loss', loss.item(), epoch)
            #writer.add_scalar('Accuracy/Accuracy', accuracy, epoch)
    scheduler.step()

In [6]:
def validate(epoch):
    model.eval()
    #writer = SummaryWriter()
    valid_loss = 0
    correct = 0
    for data, target in valid_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        valid_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()

    valid_loss /= len(valid_idx)
    accuracy = 100. * correct.cpu().numpy() / len(valid_idx)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        valid_loss, correct, len(valid_idx),
        100. * correct / len(valid_idx)))
    #writer.add_scalar('Loss/Validation_Loss', valid_loss, epoch)
    #writer.add_scalar('Accuracy/Validation_Accuracy', accuracy, epoch)
    return valid_loss, accuracy

In [7]:
# Fix best model

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct.cpu().numpy() / len(test_loader.dataset)))

In [8]:
def save_best(loss, accuracy, best_loss, best_acc):
    if best_loss == None:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
        
    elif loss < best_loss and accuracy > best_acc:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
    return best_loss, best_acc

In [9]:
# Fantastic logger for tensorboard and pytorch, 
# run tensorboard by opening a new terminal and run "tensorboard --logdir runs"
# open tensorboard at http://localhost:6006/
from tensorboardX import SummaryWriter
best_loss = None
best_acc = None

import time 
SINCE=time.time()

for epoch in range(max_epoch):
    train(epoch)
    loss, accuracy = validate(epoch)
    best_loss, best_acc = save_best(loss, accuracy, best_loss, best_acc)
    
    NOW=time.time() 
    DURINGS=NOW-SINCE
    SINCE=NOW
    print("the time of this epoch:[{} s]".format(DURINGS))
    
#writer = SummaryWriter()
#writer.export_scalars_to_json("./all_scalars.json")

#writer.close()

#---------------------------- Test ------------------------------
test(epoch)

tensor(886.9125, device='cuda:0')
torch.Size([1024, 1, 3])
Parameter containing:
tensor([[[ 0.1876,  0.3725,  0.4909]],

        [[ 0.0057,  0.2638, -0.2336]],

        [[-0.3289, -0.1337,  0.1112]],

        ...,

        [[ 0.2788,  0.4954,  0.5582]],

        [[ 0.1081, -0.3743, -0.0758]],

        [[-0.4959,  0.2686,  0.0928]]], device='cuda:0')
Train Epoch: 0 [0/50000 (0%)]	Loss: 2.303099, Accuracy: 9.77
Train Epoch: 0 [2560/50000 (6%)]	Loss: 2.623101, Accuracy: 7.03
Train Epoch: 0 [5120/50000 (11%)]	Loss: 2.307165, Accuracy: 11.33
Train Epoch: 0 [7680/50000 (17%)]	Loss: 2.332282, Accuracy: 8.20
Train Epoch: 0 [10240/50000 (23%)]	Loss: 2.305197, Accuracy: 11.33
Train Epoch: 0 [12800/50000 (28%)]	Loss: 2.342893, Accuracy: 9.38
Train Epoch: 0 [15360/50000 (34%)]	Loss: 2.230751, Accuracy: 16.02
Train Epoch: 0 [17920/50000 (40%)]	Loss: 2.146911, Accuracy: 17.58
tensor(913.0258, device='cuda:0')
torch.Size([1024, 1, 3])
Parameter containing:
tensor([[[ 2.2267e-01,  3.6848e-01,  5.0781e

Train Epoch: 3 [23040/50000 (51%)]	Loss: 1.672054, Accuracy: 33.59
Train Epoch: 3 [25600/50000 (57%)]	Loss: 1.798072, Accuracy: 26.17
Train Epoch: 3 [28160/50000 (62%)]	Loss: 1.758943, Accuracy: 29.69
Train Epoch: 3 [30720/50000 (68%)]	Loss: 1.800080, Accuracy: 29.30
Train Epoch: 3 [33280/50000 (74%)]	Loss: 1.836377, Accuracy: 28.52
Train Epoch: 3 [35840/50000 (80%)]	Loss: 1.790584, Accuracy: 23.83
Train Epoch: 3 [38400/50000 (85%)]	Loss: 1.766932, Accuracy: 23.44
tensor(1001.8877, device='cuda:0')
torch.Size([1024, 1, 3])
Parameter containing:
tensor([[[ 0.4990,  0.2456,  0.5164]],

        [[ 0.0877,  0.3727, -0.1217]],

        [[-0.4172, -0.0833,  0.0902]],

        ...,

        [[ 0.2978,  0.5420,  0.6358]],

        [[ 0.1589, -0.4987, -0.1945]],

        [[-0.4166,  0.3629,  0.0790]]], device='cuda:0')
Train Epoch: 3 [40960/50000 (91%)]	Loss: 1.729364, Accuracy: 30.08
Train Epoch: 3 [43520/50000 (97%)]	Loss: 1.695360, Accuracy: 27.34

Validation set: Average loss: 1.7666, Accur


Validation set: Average loss: 1.6627, Accuracy: 1899/5000 (37.00%)

the time of this epoch:[154.97862839698792 s]
tensor(1134.0073, device='cuda:0')
torch.Size([1024, 1, 3])
Parameter containing:
tensor([[[ 5.3212e-01,  7.5836e-02,  8.6666e-01]],

        [[ 8.7704e-02,  3.7274e-01, -1.2173e-01]],

        [[-4.2128e-01, -1.0283e-01,  8.6048e-02]],

        ...,

        [[ 2.9174e-01,  6.4943e-01,  7.1858e-01]],

        [[ 2.0217e-01, -6.9422e-01, -1.1530e-01]],

        [[-4.1657e-01,  3.6286e-01,  7.8983e-02]]], device='cuda:0')
Train Epoch: 7 [0/50000 (0%)]	Loss: 1.631681, Accuracy: 41.41
Train Epoch: 7 [2560/50000 (6%)]	Loss: 1.617286, Accuracy: 35.16
Train Epoch: 7 [5120/50000 (11%)]	Loss: 1.625930, Accuracy: 37.50
Train Epoch: 7 [7680/50000 (17%)]	Loss: 1.547817, Accuracy: 38.67
Train Epoch: 7 [10240/50000 (23%)]	Loss: 1.599638, Accuracy: 42.97
Train Epoch: 7 [12800/50000 (28%)]	Loss: 1.590761, Accuracy: 41.02
Train Epoch: 7 [15360/50000 (34%)]	Loss: 1.639710, Accuracy: 36.33


Train Epoch: 10 [15360/50000 (34%)]	Loss: 1.329424, Accuracy: 51.95
Train Epoch: 10 [17920/50000 (40%)]	Loss: 1.342193, Accuracy: 50.78
tensor(1290.4641, device='cuda:0')
torch.Size([1024, 1, 3])
Parameter containing:
tensor([[[ 0.3834,  0.1829,  1.0896]],

        [[ 0.0877,  0.3727, -0.1217]],

        [[-0.2578, -0.4614,  0.1668]],

        ...,

        [[ 0.0485,  0.9192,  0.6606]],

        [[ 0.1411, -0.8675, -0.2887]],

        [[-0.4166,  0.3629,  0.0790]]], device='cuda:0')
Train Epoch: 10 [20480/50000 (45%)]	Loss: 1.277708, Accuracy: 55.47
Train Epoch: 10 [23040/50000 (51%)]	Loss: 1.305436, Accuracy: 51.17
Train Epoch: 10 [25600/50000 (57%)]	Loss: 1.188051, Accuracy: 60.55
Train Epoch: 10 [28160/50000 (62%)]	Loss: 1.298064, Accuracy: 49.61
Train Epoch: 10 [30720/50000 (68%)]	Loss: 1.380910, Accuracy: 51.56
Train Epoch: 10 [33280/50000 (74%)]	Loss: 1.123533, Accuracy: 57.42
Train Epoch: 10 [35840/50000 (80%)]	Loss: 1.279999, Accuracy: 57.42
Train Epoch: 10 [38400/50000 (85%)]

Train Epoch: 13 [38400/50000 (85%)]	Loss: 0.997473, Accuracy: 65.23
tensor(1342.6089, device='cuda:0')
torch.Size([1024, 1, 3])
Parameter containing:
tensor([[[ 2.8755e-01,  3.3931e-01,  1.0750e+00]],

        [[ 8.7704e-02,  3.7274e-01, -1.2173e-01]],

        [[-9.9666e-02, -6.4661e-01, -1.1496e-01]],

        ...,

        [[-5.1840e-02,  1.0801e+00,  4.8459e-01]],

        [[ 1.4770e-01, -9.7942e-01, -2.2372e-01]],

        [[-4.1657e-01,  3.6286e-01,  7.8983e-02]]], device='cuda:0')
Train Epoch: 13 [40960/50000 (91%)]	Loss: 1.302253, Accuracy: 59.38
Train Epoch: 13 [43520/50000 (97%)]	Loss: 1.094311, Accuracy: 61.72

Validation set: Average loss: 1.1111, Accuracy: 3142/5000 (62.00%)

the time of this epoch:[154.79829597473145 s]
tensor(1342.9496, device='cuda:0')
torch.Size([1024, 1, 3])
Parameter containing:
tensor([[[ 0.3076,  0.3336,  1.0677]],

        [[ 0.0877,  0.3727, -0.1217]],

        [[-0.1073, -0.6423, -0.1244]],

        ...,

        [[-0.0434,  1.0960,  0.4643]],



...

Train Epoch: 44 [12800/50000 (28%)]	Loss: 0.457516, Accuracy: 85.16
Train Epoch: 44 [15360/50000 (34%)]	Loss: 0.499136, Accuracy: 80.86
Train Epoch: 44 [17920/50000 (40%)]	Loss: 0.496809, Accuracy: 86.72
tensor(1456.8606, device='cuda:0')
torch.Size([1024, 1, 3])
Parameter containing:
tensor([[[ 0.2993,  0.1652,  1.2148]],

        [[ 0.0877,  0.3727, -0.1217]],

        [[-0.3948, -0.6932, -0.1667]],

        ...,

        [[-0.1209,  1.3139,  0.3271]],

        [[ 0.0425, -1.1506, -0.2439]],

        [[-0.4166,  0.3629,  0.0790]]], device='cuda:0')
Train Epoch: 44 [20480/50000 (45%)]	Loss: 0.500593, Accuracy: 85.55
Train Epoch: 44 [23040/50000 (51%)]	Loss: 0.493758, Accuracy: 86.33
Train Epoch: 44 [25600/50000 (57%)]	Loss: 0.504778, Accuracy: 85.55
Train Epoch: 44 [28160/50000 (62%)]	Loss: 0.529989, Accuracy: 85.16
Train Epoch: 44 [30720/50000 (68%)]	Loss: 0.516130, Accuracy: 85.55
Train Epoch: 44 [33280/50000 (74%)]	Loss: 0.483525, Accuracy: 86.72
Train Epoch: 44 [35840/50000 (80%)]

Train Epoch: 47 [33280/50000 (74%)]	Loss: 0.401832, Accuracy: 86.33
Train Epoch: 47 [35840/50000 (80%)]	Loss: 0.560707, Accuracy: 81.25
Train Epoch: 47 [38400/50000 (85%)]	Loss: 0.482633, Accuracy: 85.16
tensor(1457.5491, device='cuda:0')
torch.Size([1024, 1, 3])
Parameter containing:
tensor([[[ 0.2932,  0.1643,  1.2192]],

        [[ 0.0877,  0.3727, -0.1217]],

        [[-0.3926, -0.6842, -0.1951]],

        ...,

        [[-0.1110,  1.3231,  0.3228]],

        [[ 0.0340, -1.1512, -0.2416]],

        [[-0.4166,  0.3629,  0.0790]]], device='cuda:0')
Train Epoch: 47 [40960/50000 (91%)]	Loss: 0.479345, Accuracy: 82.81
Train Epoch: 47 [43520/50000 (97%)]	Loss: 0.561534, Accuracy: 83.59

Validation set: Average loss: 0.5975, Accuracy: 4162/5000 (83.00%)

the time of this epoch:[154.79656529426575 s]
tensor(1457.5946, device='cuda:0')
torch.Size([1024, 1, 3])
Parameter containing:
tensor([[[ 0.2948,  0.1652,  1.2174]],

        [[ 0.0877,  0.3727, -0.1217]],

        [[-0.3922, -0.6849, -0

Train Epoch: 51 [2560/50000 (6%)]	Loss: 0.457774, Accuracy: 87.50
Train Epoch: 51 [5120/50000 (11%)]	Loss: 0.560232, Accuracy: 83.20
Train Epoch: 51 [7680/50000 (17%)]	Loss: 0.491905, Accuracy: 84.77
Train Epoch: 51 [10240/50000 (23%)]	Loss: 0.459941, Accuracy: 86.72
Train Epoch: 51 [12800/50000 (28%)]	Loss: 0.513132, Accuracy: 82.81
Train Epoch: 51 [15360/50000 (34%)]	Loss: 0.531609, Accuracy: 85.16
Train Epoch: 51 [17920/50000 (40%)]	Loss: 0.523849, Accuracy: 84.77
tensor(1458.6729, device='cuda:0')
torch.Size([1024, 1, 3])
Parameter containing:
tensor([[[ 0.2938,  0.1623,  1.2184]],

        [[ 0.0877,  0.3727, -0.1217]],

        [[-0.3864, -0.6915, -0.1927]],

        ...,

        [[-0.1139,  1.3300,  0.3162]],

        [[ 0.0346, -1.1480, -0.2463]],

        [[-0.4166,  0.3629,  0.0790]]], device='cuda:0')
Train Epoch: 51 [20480/50000 (45%)]	Loss: 0.499564, Accuracy: 86.33
Train Epoch: 51 [23040/50000 (51%)]	Loss: 0.407574, Accuracy: 85.94
Train Epoch: 51 [25600/50000 (57%)]	Los

Train Epoch: 54 [23040/50000 (51%)]	Loss: 0.468098, Accuracy: 85.16
Train Epoch: 54 [25600/50000 (57%)]	Loss: 0.471298, Accuracy: 86.33
Train Epoch: 54 [28160/50000 (62%)]	Loss: 0.449545, Accuracy: 88.67
Train Epoch: 54 [30720/50000 (68%)]	Loss: 0.488713, Accuracy: 84.38
Train Epoch: 54 [33280/50000 (74%)]	Loss: 0.381005, Accuracy: 87.50
Train Epoch: 54 [35840/50000 (80%)]	Loss: 0.505362, Accuracy: 85.55
Train Epoch: 54 [38400/50000 (85%)]	Loss: 0.443500, Accuracy: 88.28
tensor(1458.8630, device='cuda:0')
torch.Size([1024, 1, 3])
Parameter containing:
tensor([[[ 0.2903,  0.1684,  1.2171]],

        [[ 0.0877,  0.3727, -0.1217]],

        [[-0.3887, -0.6891, -0.1942]],

        ...,

        [[-0.1189,  1.3321,  0.3128]],

        [[ 0.0296, -1.1467, -0.2471]],

        [[-0.4166,  0.3629,  0.0790]]], device='cuda:0')
Train Epoch: 54 [40960/50000 (91%)]	Loss: 0.470385, Accuracy: 84.38
Train Epoch: 54 [43520/50000 (97%)]	Loss: 0.458498, Accuracy: 85.55

Validation set: Average loss: 0.58

tensor(1459.2019, device='cuda:0')
torch.Size([1024, 1, 3])
Parameter containing:
tensor([[[ 0.2888,  0.1686,  1.2197]],

        [[ 0.0877,  0.3727, -0.1217]],

        [[-0.3794, -0.6954, -0.1992]],

        ...,

        [[-0.1370,  1.3255,  0.3117]],

        [[ 0.0263, -1.1545, -0.2390]],

        [[-0.4166,  0.3629,  0.0790]]], device='cuda:0')
Train Epoch: 58 [0/50000 (0%)]	Loss: 0.527793, Accuracy: 85.55
Train Epoch: 58 [2560/50000 (6%)]	Loss: 0.441032, Accuracy: 87.50
Train Epoch: 58 [5120/50000 (11%)]	Loss: 0.414081, Accuracy: 87.50
Train Epoch: 58 [7680/50000 (17%)]	Loss: 0.521524, Accuracy: 85.94
Train Epoch: 58 [10240/50000 (23%)]	Loss: 0.380536, Accuracy: 88.67
Train Epoch: 58 [12800/50000 (28%)]	Loss: 0.478274, Accuracy: 85.16
Train Epoch: 58 [15360/50000 (34%)]	Loss: 0.406828, Accuracy: 88.28
Train Epoch: 58 [17920/50000 (40%)]	Loss: 0.530619, Accuracy: 84.77
tensor(1459.3094, device='cuda:0')
torch.Size([1024, 1, 3])
Parameter containing:
tensor([[[ 0.2885,  0.1666,  1

# Step 3: Test

In [12]:
test(epoch)


Test set: Average loss: 0.4542, Accuracy: 8932/10000 (89.32%)

